# DataWrangling, Graphical & Non-graphical EDA methods to fix data problems for Transactional Retail data 



## Table of Content
1. [Dirty Data](#1)
2. [Outlier Data](#2)
3. [Missing Data](#3)

In [1]:
#Import libraries

#for dataframe manipulation
import pandas as pd
from pandas import DataFrame

#For sentiment analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#For plotting graph
import matplotlib.pyplot as plt
%matplotlib inline 

#For linear regression model
from sklearn.linear_model import LinearRegression

#To perform mathematical operation
from math import radians, cos, sin, asin, sqrt

#To manipulate dataframe
import numpy as np

## 1. Dirty Data <a class="anchor" id="1"></a>

`Dirty_data` data file contains various errors and data quality issues. I will identify all of them and fix them accordingly.

In [10]:
#Load Dirty_data 
dirty_data=pd.read_csv('Dirty_data.csv')
dirty_data.head()

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
0,ORD377463,ID3743608254,2019-06-20,Thompson,"[('Universe Note', 2), ('Lucent 330S', 1), ('C...",17510,51.19,-37.822199,144.947623,5,16685.69,Winter,False,1.0615,just buy a $10 android phone instead what a pi...,False
1,ORD482178,ID4716795332,2019-11-20,Thompson,"[('Lucent 330S', 2), ('Olivia x460', 1), ('iAs...",8435,102.64,-37.819283,144.952701,10,7694.14,Spring,True,0.8870,i own the first and second generation thunder ...,True
2,ORD147079,ID1224821948,2019-11-17,nickolson,"[('Candle Inferno', 2), ('iAssist Line', 1), (...",14175,98.82,-37.812082,144.938374,5,13565.07,Spring,True,0.7675,most people will go with a bigger more sofisti...,True
3,ORD006037,ID3240572176,2019-07-26,Thompson,"[('Thunder line', 1), ('Universe Note', 1), ('...",9085,68.83,-37.805653,144.930917,25,6882.58,Winter,False,0.9984,**positive review** good buy great price and m...,True
4,ORD472716,ID0123087089,2019-04-28,Thompson,"[('iAssist Line', 1), ('Olivia x460', 1)]",3450,78.83,-37.814832,144.955431,25,2666.33,Autumn,True,0.7736,"mint condition phone is brand new, only minimu...",True


In [3]:
dirty_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       500 non-null    object 
 1   customer_id                    500 non-null    object 
 2   date                           500 non-null    object 
 3   nearest_warehouse              500 non-null    object 
 4   shopping_cart                  500 non-null    object 
 5   order_price                    500 non-null    int64  
 6   delivery_charges               500 non-null    float64
 7   customer_lat                   500 non-null    float64
 8   customer_long                  500 non-null    float64
 9   coupon_discount                500 non-null    int64  
 10  order_total                    500 non-null    float64
 11  season                         500 non-null    object 
 12  is_expedited_delivery          500 non-null    boo

`dirty_data` contains 500 rows of data, with 16 attributes/columns. There does not seem to be any missing values. Most of the attributes have the correct datatypes. Next, I will closely examine each individual attribute/column to see if there are any errors. According to business rules, there are some relationship between attributes, so I will take that into consideration when identifying the errors as well.

### Examine `order_id`

`order_id` consists of a string `ORD`, followed by 6 digits. Let's check if all the values in `order_id` follows this pattern.

In [11]:
dirty_data[dirty_data['order_id'].str.match('ORD\d{6}')==False]

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer


All values in `order_id` have the correct format

In [12]:
len(dirty_data['order_id'].unique())
#There are no duplicated order_id, which should be unique

500

### Examine `customer_id`

`customer_id` consists of a string `ID`, followed by 10 digits. Let's check if all the values in `customer_id` follows this pattern

In [13]:
dirty_data[dirty_data['customer_id'].str.match('ID\d{10}')==False]

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer


All values in `customer_id` have the correct format

In [14]:
len(dirty_data['customer_id'].unique())

493

There are duplicated values in `customer_id`. However, this is acceptable because some customers have more than 1 order, and it is normal for 1 customer to have multiple orders. Even though `order_id` must be unique, `customer_id` does not need to be. 

### Examine `data`

`date` all has the year of 2019, followed by month, and then day, separated by dash `-`. Basically, the general pattern is `2019-mm-dd`. 

In [15]:
#Check if there are any values in date that do not follow the above pattern
date_pattern=r'''(?x)
    #year
    2019-
    #February (28 days since it is 2019)
    ((02)-([0][1-9]|[1][0-9]|[2][0-8])
    #30-day months
    |(0[469]|11)-([0][1-9]|[12][0-9]|30)
    #31-day months
    |(0[13578]|1[02])-([0][1-9]|[12][0-9]|3[01]))
'''

dirty_date=dirty_data[dirty_data['date'].str.match(date_pattern)==False]
dirty_date

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
8,ORD162047,ID0364845517,2019-26-02,Bakers,"[('pearTV', 1), ('Universe Note', 1), ('iStrea...",11290,101.06,-37.801250,144.980627,25,8568.56,Summer,True,1.6118,"great purchase!! great phone, came unlocked in...",True
17,ORD097772,ID4716795186,2019-30-07,Thompson,"[('iAssist Line', 2), ('Thunder line', 1), ('U...",10940,65.40,-37.819244,144.951841,15,9364.40,Winter,False,0.8433,works great for everything we want our daughte...,True
18,ORD116319,ID1131986160,08-09-2019,Thompson,"[('Candle Inferno', 2), ('pearTV', 1), ('Olivi...",9620,96.22,-37.815221,144.945023,15,8273.22,Spring,True,0.3359,so good i bought it twice! after sacrificing m...,True
23,ORD107290,ID0634784120,12-11-2019,Thompson,"[('iAssist Line', 2), ('Olivia x460', 2), ('To...",15540,78.12,-37.813974,144.937757,10,14064.12,Spring,False,0.8317,phone is excellent but does not come with sd c...,True
65,ORD363505,ID4418536073,04-06-2019,Thompson,"[('iAssist Line', 2), ('pearTV', 1), ('Candle ...",11190,67.47,-37.802092,144.963514,0,11257.47,Winter,True,1.8653,one star i am not happy with this phone. low b...,False
101,ORD351928,ID6167413262,02-11-2019,Nickolson,"[('Toshika 750', 1), ('Thunder line', 2), ('iS...",8830,74.10,-37.813649,144.965578,5,8462.60,Spring,False,0.6520,five stars good phone,True
117,ORD086478,ID0257506562,07-12-2019,Thompson,"[('Candle Inferno', 1), ('Lucent 330S', 1), ('...",11525,85.42,-37.799728,144.963010,15,9881.67,Summer,False,2.0105,"extremely good value for the money, xp 6.1 is ...",True
121,ORD022904,ID2209383610,03-08-2019,Thompson,"[('iAssist Line', 2), ('iStream', 1)]",4600,83.76,-37.799680,144.957501,25,3533.76,Winter,True,1.7128,five stars it is running well! delivered as in...,True
122,ORD062086,ID0589500304,2019-14-04,Nickolson,"[('Thunder line', 1), ('Candle Inferno', 2)]",3040,65.07,-37.820228,144.957411,10,2801.07,Autumn,False,1.0829,they are very happy with the phone got these a...,True
145,ORD198966,ID0255058361,10-05-2019,Nickolson,"[('iAssist Line', 2), ('Toshika 750', 2), ('Un...",20290,63.71,-37.812879,144.971604,15,17310.21,Autumn,False,0.6614,great seller great product. just received it. ...,True


In [17]:
len(dirty_date)

27

There are 27 values in `date` that are not in the correct format. They are either in the format `2019-dd-mm` or `dd-mm-2019`. Fix them by converting them into the correct format `2019-mm-dd`. From here onwards, we will also create a dataframe that does not contain any detected anomalies yet SO FAR called `maybe_clean_data`

In [18]:
maybe_clean_data=dirty_data.drop(dirty_date.index,axis=0)

In [19]:
#Those dates that are in format 2019-dd-mm
wrong_date_pattern_1=r'''(?x)
        #year
        2019-
        #February (28 days since it is 2019)
        (([0][1-9]|[1][0-9]|[2][0-8]-(02))
        #30-day months
        |([0][1-9]|[12][0-9]|30-(0[469]|11))
        #31-day months
        |([0][1-9]|[12][0-9]|3[01])-(0[13578]|1[02]))
'''
wrong_date_1=dirty_date[dirty_date["date"].str.match(wrong_date_pattern_1)==True]
wrong_date_1

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
8,ORD162047,ID0364845517,2019-26-02,Bakers,"[('pearTV', 1), ('Universe Note', 1), ('iStrea...",11290,101.06,-37.801250,144.980627,25,8568.56,Summer,True,1.6118,"great purchase!! great phone, came unlocked in...",True
17,ORD097772,ID4716795186,2019-30-07,Thompson,"[('iAssist Line', 2), ('Thunder line', 1), ('U...",10940,65.40,-37.819244,144.951841,15,9364.40,Winter,False,0.8433,works great for everything we want our daughte...,True
122,ORD062086,ID0589500304,2019-14-04,Nickolson,"[('Thunder line', 1), ('Candle Inferno', 2)]",3040,65.07,-37.820228,144.957411,10,2801.07,Autumn,False,1.0829,they are very happy with the phone got these a...,True
175,ORD418056,ID0092149939,2019-27-03,Nickolson,"[('Candle Inferno', 1), ('pearTV', 1), ('Olivi...",9190,66.46,-37.822327,144.956400,25,6958.96,Autumn,False,1.2288,love it great phone. my second s8 active. thes...,True
213,ORD213633,ID0289602682,2019-26-01,Thompson,"[('Universe Note', 2), ('iStream', 1), ('Lucen...",18150,69.86,-37.813418,144.950358,0,18219.86,Summer,False,0.3009,nice phone i bought this phone for my husband ...,True
235,ORD195454,ID6197209818,2019-27-09,Thompson,"[('Thunder line', 1), ('iStream', 1), ('pearTV...",14950,106.62,-37.817973,144.932053,10,13561.62,Spring,True,1.4463,this is great for an 8 yr old. she had out gro...,True
243,ORD060799,ID5058108344,2019-24-08,Nickolson,"[('Candle Inferno', 2), ('iStream', 2), ('pear...",15005,46.79,-37.822472,144.969411,0,15051.79,Winter,False,0.4317,phone had a problem phone seemed great but con...,False
250,ORD464628,ID3157171390,2019-26-12,Thompson,"[('iStream', 1), ('iAssist Line', 2)]",4600,76.89,-37.821724,144.955481,10,4216.89,Summer,False,1.2499,locked to verizon it's nice and new but its lo...,True
277,ORD334604,ID4271393503,2019-29-01,Nickolson,"[('Lucent 330S', 2), ('Alcon 10', 1), ('Thunde...",13590,96.36,-37.815220,144.959260,0,13686.36,Summer,True,0.9798,excellent basic phone even in 2019 i got this ...,True
307,ORD077824,ID0137148242,2019-16-10,Thompson,"[('iStream', 2), ('Alcon 10', 2)]",18200,82.74,-37.799833,144.939835,10,16462.74,Spring,False,1.5645,great purches great phone works perfect. it wa...,True


In [22]:
#Make a list of those wrongly formatted dates
date_list_1=list(wrong_date_1['date'])
date_list_1

['2019-26-02',
 '2019-30-07',
 '2019-14-04',
 '2019-27-03',
 '2019-26-01',
 '2019-27-09',
 '2019-24-08',
 '2019-26-12',
 '2019-29-01',
 '2019-16-10',
 '2019-26-04',
 '2019-30-01',
 '2019-29-10',
 '2019-22-04']

In [23]:
#Change the position of month and date so we get the correct format
for i in range(len(date_list_1)):
    day=date_list_1[i][5:7]
    month=date_list_1[i][-2:]
    date_list_1[i]='2019-'+month+'-'+day
date_list_1

['2019-02-26',
 '2019-07-30',
 '2019-04-14',
 '2019-03-27',
 '2019-01-26',
 '2019-09-27',
 '2019-08-24',
 '2019-12-26',
 '2019-01-29',
 '2019-10-16',
 '2019-04-26',
 '2019-01-30',
 '2019-10-29',
 '2019-04-22']

In [24]:
#Convert the list into a dataframe
date_df_1=DataFrame(date_list_1)

#Replace those wrongly formatted dates in wrong_date_1 with the correct ones
wrong_date_1['date']=date_df_1[0].values

#Then replace those dates in dirty_data that were in 2019-dd-mm format with the correct ones
dirty_data.loc[list(wrong_date_1.index),'date']=wrong_date_1['date'].values

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [25]:
#Now we check again for those remaining dates that are still not in the correct format 2019-mm-dd
wrong_date_2=dirty_data[dirty_data['date'].str.match(date_pattern)==False]
wrong_date_2

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer
18,ORD116319,ID1131986160,08-09-2019,Thompson,"[('Candle Inferno', 2), ('pearTV', 1), ('Olivi...",9620,96.22,-37.815221,144.945023,15,8273.22,Spring,True,0.3359,so good i bought it twice! after sacrificing m...,True
23,ORD107290,ID0634784120,12-11-2019,Thompson,"[('iAssist Line', 2), ('Olivia x460', 2), ('To...",15540,78.12,-37.813974,144.937757,10,14064.12,Spring,False,0.8317,phone is excellent but does not come with sd c...,True
65,ORD363505,ID4418536073,04-06-2019,Thompson,"[('iAssist Line', 2), ('pearTV', 1), ('Candle ...",11190,67.47,-37.802092,144.963514,0,11257.47,Winter,True,1.8653,one star i am not happy with this phone. low b...,False
101,ORD351928,ID6167413262,02-11-2019,Nickolson,"[('Toshika 750', 1), ('Thunder line', 2), ('iS...",8830,74.10,-37.813649,144.965578,5,8462.60,Spring,False,0.6520,five stars good phone,True
117,ORD086478,ID0257506562,07-12-2019,Thompson,"[('Candle Inferno', 1), ('Lucent 330S', 1), ('...",11525,85.42,-37.799728,144.963010,15,9881.67,Summer,False,2.0105,"extremely good value for the money, xp 6.1 is ...",True
121,ORD022904,ID2209383610,03-08-2019,Thompson,"[('iAssist Line', 2), ('iStream', 1)]",4600,83.76,-37.799680,144.957501,25,3533.76,Winter,True,1.7128,five stars it is running well! delivered as in...,True
145,ORD198966,ID0255058361,10-05-2019,Nickolson,"[('iAssist Line', 2), ('Toshika 750', 2), ('Un...",20290,63.71,-37.812879,144.971604,15,17310.21,Autumn,False,0.6614,great seller great product. just received it. ...,True
157,ORD032235,ID2288769478,04-02-2019,Bakers,"[('Candle Inferno', 1), ('iStream', 1), ('Thun...",2760,78.09,-37.801670,145.003551,5,2700.09,Summer,False,1.1808,juggernaut love this phone. options and use is...,True
184,ORD223648,ID0130015101,06-06-2019,Nickolson,"[('Universe Note', 2), ('pearTV', 1)]",13210,57.48,-37.815550,144.967998,0,13267.48,Winter,True,0.3655,box only! the cellphone was not found!,False
238,ORD310767,ID0214092481,01-12-2019,Bakers,"[('Olivia x460', 1), ('Thunder line', 2), ('Lu...",8045,82.72,-37.805250,145.015173,10,7323.22,Summer,False,1.8316,love it. gr8 phone.,True


In [26]:
len(wrong_date_2)

13

Now we need to take care of these 13 dates that are in the wrong format of `dd-mm-2019`

In [30]:
#Make a list of these wrongly formatted dates
date_list_2=list(wrong_date_2['date'])
date_list_2

['08-09-2019',
 '12-11-2019',
 '04-06-2019',
 '02-11-2019',
 '07-12-2019',
 '03-08-2019',
 '10-05-2019',
 '04-02-2019',
 '06-06-2019',
 '01-12-2019',
 '12-08-2019',
 '01-09-2019',
 '01-03-2019']

In [31]:
#Change the position of year and date so we get the correct format
for i in range(len(date_list_2)):
    day=date_list_2[i][:2]
    month=date_list_2[i][3:5]
    date_list_2[i]='2019-'+month+'-'+day
date_list_2

['2019-09-08',
 '2019-11-12',
 '2019-06-04',
 '2019-11-02',
 '2019-12-07',
 '2019-08-03',
 '2019-05-10',
 '2019-02-04',
 '2019-06-06',
 '2019-12-01',
 '2019-08-12',
 '2019-09-01',
 '2019-03-01']

In [32]:
#Convert into a dataframe
date_df_2=DataFrame(date_list_2)

#Replace those wrongly formatted dates in wrong_date_2 with the correct ones
wrong_date_2['date']=date_df_2[0].values

#Then replace those dates in dirty_data that are in dd-mm-2019 format with the correct ones
dirty_data.loc[list(wrong_date_2.index),'date']=wrong_date_2['date'].values

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [33]:
#Now we check again if there are any discrepancy in date
dirty_data[dirty_data['date'].str.match(date_pattern)==False]

,order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer


All the values in `date` column are fixed now

In [34]:
#Convert to correct datetime data type
dirty_data['date']=pd.to_datetime(dirty_data['date'])

### Examine `nearest_warehouse`

In [37]:
dirty_data['nearest_warehouse'].unique()

array(['Thompson', 'nickolson', 'Nickolson', 'Bakers', 'bakers',
       'thompson'], dtype=object)

According to business rules, the retail store only has 3 different warehouses, but here we have 6 unique values. However, they just seem like an inconsistency in formatting (uppercase vs lowercase). To check for the correct format of values of `nearest_warehouse` attribute, we load `warehouses` dataset and check

In [38]:
warehouses=pd.read_csv('warehouses.csv')
warehouses

,names,lat,lon
0,Nickolson,-37.818595,144.969551
1,Thompson,-37.812673,144.947069
2,Bakers,-37.809996,144.995232


As we can see, the uppercase is the correct version, hence we will make the corresponding changes. We also update `maybe_clean_data` by removing rows that have the wrong format of `nearest_warehouse`

In [45]:
#Get a list of values of nearest_warehouse that do not have the first letter as uppercase
lowercase_name=[i for i in dirty_data['nearest_warehouse'].unique().tolist() if i[0].isupper()==False]

In [50]:
wrong_warehouse=dirty_data[dirty_data['nearest_warehouse'].isin(lowercase_name)]

#Update maybe_clean_data
maybe_clean_data=maybe_clean_data.drop(wrong_warehouse.index,axis=0)

In [63]:
#Convert to correct formats, by capitalizing the first letter
dirty_data['nearest_warehouse']=dirty_data['nearest_warehouse'].apply(lambda x: x.capitalize())

In [64]:
dirty_data['nearest_warehouse'].unique()
#No more inconsistencies in formatting

array(['Thompson', 'Nickolson', 'Bakers'], dtype=object)

### Examine `delivery_charges` and `coupon_discount`

According to business rules, `delivery_charges` and `coupon_discount` are all correct, without any errors or data quality issues.

### Examine `customer_lat` and `customer_long`

## 2. Outlier Data <a class="anchor" id="2"></a>

## 3. Missing Data <a class="anchor" id="3"></a>